## BitsandBytes

In [1]:
###########################
import os
def gpu_info():
    dat=[]
    import subprocess
    out = subprocess.run(["nvidia-smi"], capture_output=True, text=True).stdout
    out2 = out.split("\n")[8:]
    splitter = out2[3]
    out3 = "\n".join(out2).split(splitter)[:-1]

    # Check GPU usage
    for idx,x in enumerate(out3):
        gpu = ((x.split("|"))[1].replace("Off", ""))
        t = ((x.split("|"))[6].replace("Off", "").replace("MiB", "").split("/"))
        ans = (int(t[0]) / int(t[1])) * 100
        dat.append({
                "gpu_id": idx,
                "gpu": ((gpu[:-5])[6:]).strip(),
                "usage": f"{ans:.2f}"
            })
    return dat
gpu_info_ans = gpu_info()
flag=0
for gpu in gpu_info_ans:
    if float(gpu['usage'])<10: 
        empty_gpu = gpu['gpu_id']
        flag=1
        break
if flag==0:
    print("No empty GPU found, please check your GPU usage.")
    exit(1)
else:
    print(f"Using GPU {empty_gpu} for processing.")

os.environ["CUDA_VISIBLE_DEVICES"] = str(empty_gpu)
############################

Using GPU 5 for processing.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # reduces memory usage (by ~8x compared to FP32).
    bnb_4bit_compute_dtype=torch.float16, # perform internal math operations (like attention, matrix multiplications) in float16 precision.
    bnb_4bit_use_double_quant=True, # use double quantization to reduce memory usage further.(8bit then 4bit)
    bnb_4bit_quant_type="nf4", # NormalFloat4 — designed to retain more information with fewer bits


)

model = AutoModelForCausalLM.from_pretrained(
    "unsloth/Qwen2.5-0.5B-Instruct",
    quantization_config=bnb_config,
    device_map="auto" # automatically assigns the model to the available GPU(s) based on the environment variable CUDA_VISIBLE_DEVICES
)


/home/mshahidul/miniconda3/envs/web/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/Qwen2.5-0.5B-Instruct")

In [5]:
model.eval()

# Prepare the prompt
prompt = "Explain quantum computing in simple terms."

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate response
with torch.no_grad():
    output_tokens = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id  # Avoid warning for missing pad token
    )

# Decode and print
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(output_text)

Explain quantum computing in simple terms. Quantum computing is a type of computing that uses quantum mechanics to process information, much like classical computers do.

In classical computing, data is represented by bits (0s and 1s), which can be stored and processed individually. In quantum computing, data is represented as quantum bits or qubits. When a qubit is measured, it collapses into one of two states - 0 or 1. This means that the same amount of information can be stored using more qubits than just one qubit. By doing this, quantum computers can perform many calculations at once, allowing them to solve problems that are beyond the capabilities of classical computers.

There are some key features of quantum computing that make it different from classical computing:

- Superposition: A
